### CT (mixed)데이터셋의 EDA와 전처리 과정
- 범주형/숫자형 변수 탐색, 그에맞는 전처리
- 타겟과 숫자형 변수에 대해 quantile 에 따른 이상치(outlier) 제거
- 스케일링 및 표준화

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)
print()

In [3]:
file = r"cycletime_train.csv"
ct_df = pd.read_csv(file)
ct_df.head(3)

C:\Users\bmseo\AppData\Local\Temp\ipykernel_6356\3477394120.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  ct_df = pd.read_csv(file)


,DR,PROCESS_ID,PART_ID,LOT_ID,LOTTYPE,QTY,HOLD_FLAG,FLOORID,GRADE,COMPLETE_RATE,LOT_PURPOSE,DUE_DATE,WIPTURN,ROOM,PROCESS_GROUP,STEP_ID,LAYER_TITLE,LAYER_GROUP,BLOCK_GROUP,PRE_STEPGBN2,EQPTYPE,MOVE_0,EQP_WORKLOAD_0,MOVE_1,EQP_WORKLOAD_1,MOVE_2,EQP_WORKLOAD_2,BATCH,Q_E,Q_P,L_E,L_P,Q_H,Q_R,Q_W,L_H,L_R,L_W,Q_2,Q_3,Q_4,Q_5,Q_7,Q_8,Q_10,L_2,L_3,L_4,L_5,L_7,L_8,L_10,WIP_WAITTIME,WIP_PRODUCT_NUM,SHIFT_TYPE,PROCESSTIME,PRC_WAIT_MIN,PRC_WAIT_MAX,PRC_WAIT_MEDIAN,PRC_WAIT_STD,WAITTIME
0,N4,BU,5P9865A02,3917J9,PT,3,Y,S3,NaN,0.64,J,NaN,NaN,PH,COATER_HSOH,VL532850,L1,LG61,PU6,ACI_OCD,PARCCO,28698,15.6,33190,18.0,28430,16.5,NaN,459.0,908.0,177,84,644.0,304.0,419.0,171,15,75,2.0,335.0,702.0,275.0,9.0,44.0,NaN,2,98,94,38,4,25,0,9449.00,53,WG,7.67,1.33,51449.62,8.47,728.10,107.33
1,N4,BU,5P9865A02,363Y1,PT,22,Y,S3,1.0,0.88,1,12.0,10.00,PH,COATER_HSOH,VL746200,D10,LG63,PU6,FOUP_TRANSFER,PARCCO,28698,15.6,33190,18.0,28430,16.5,NaN,489.0,906.0,174,75,747.0,196.0,452.0,169,12,68,NaN,398.0,659.0,234.0,10.0,94.0,NaN,0,96,82,38,5,28,0,10246.02,50,WS,11.87,1.33,51449.62,8.47,728.10,273.13
2,N5,LJ,FK05M2X01,383P1,PP,22,Y,S3,NaN,0.78,1,38.0,5.34,PH,COATER_HSOH,KB601700,D5,LG63,PU6,FOUP_TRANSFER,PARCCO,28698,15.6,33190,18.0,28430,16.5,NaN,489.0,906.0,174,75,747.0,196.0,452.0,169,12,68,NaN,398.0,659.0,234.0,10.0,94.0,NaN,0,96,82,38,5,28,0,10246.02,50,WS,16.03,1.63,6477.23,9.22,170.92,247.97


In [4]:
# 불필요 피쳐 삭제
ct_df.drop('LOT_ID', axis=1, inplace=True)
ct_df.drop('STEP_ID', axis=1, inplace=True)
ct_df.drop('PRE_STEPGBN2', axis=1, inplace=True)

In [5]:
# 타겟값을 waittime -> cycletime 으로 변경 
ct_df['CYCLETIME'] = ct_df['WAITTIME']+ ct_df['PROCESSTIME']
ct_df.drop('PROCESSTIME', axis=1, inplace=True)
ct_df.drop('WAITTIME', axis=1, inplace=True)

In [7]:
# ▶ 숫자형, 범주형 피쳐 리스트 생성
numerical_list=[]
categorical_list=[]

for i in ct_df.columns :
  if ct_df[i].dtypes == 'object' :
    categorical_list.append(i)
  else :
    numerical_list.append(i)

numerical_list.remove('CYCLETIME')  # 타겟은 제외

print("categorical_list {}:".format(len(categorical_list)), categorical_list)
print("numerical_list {}:".format(len(numerical_list)), numerical_list)

categorical_list 15: ['DR', 'PROCESS_ID', 'PART_ID', 'LOTTYPE', 'HOLD_FLAG', 'FLOORID', 'LOT_PURPOSE', 'ROOM', 'PROCESS_GROUP', 'LAYER_TITLE', 'LAYER_GROUP', 'BLOCK_GROUP', 'EQPTYPE', 'BATCH', 'SHIFT_TYPE']
numerical_list 41: ['QTY', 'GRADE', 'COMPLETE_RATE', 'DUE_DATE', 'WIPTURN', 'MOVE_0', 'EQP_WORKLOAD_0', 'MOVE_1', 'EQP_WORKLOAD_1', 'MOVE_2', 'EQP_WORKLOAD_2', 'Q_E', 'Q_P', 'L_E', 'L_P', 'Q_H', 'Q_R', 'Q_W', 'L_H', 'L_R', 'L_W', 'Q_2', 'Q_3', 'Q_4', 'Q_5', 'Q_7', 'Q_8', 'Q_10', 'L_2', 'L_3', 'L_4', 'L_5', 'L_7', 'L_8', 'L_10', 'WIP_WAITTIME', 'WIP_PRODUCT_NUM', 'PRC_WAIT_MIN', 'PRC_WAIT_MAX', 'PRC_WAIT_MEDIAN', 'PRC_WAIT_STD']


In [19]:
# ▶ 범주형 변수 클래스 개수 확인
list_of_df = []

for var in categorical_list :
  loop_df = pd.DataFrame({'var':[var], 'ncnt':[ct_df[var].nunique()]})
  list_of_df.append(loop_df)

df_concat = pd.concat(list_of_df).reset_index(drop=True) #reset_index 안하면 인덱스가 모두 0으로 되어있음

print(type(list_of_df[0]),list_of_df[0].shape)  # list_of_df 에는 1행2열짜리 DF가 범주형 변수 개수만큼 저장. 
df_concat.head()

<class 'pandas.core.frame.DataFrame'> (1, 2)


,var,ncnt
0,DR,4
1,PROCESS_ID,35
2,PART_ID,43
3,LOTTYPE,12
4,HOLD_FLAG,1


In [21]:
# NULL값 처리
ct_df['HOLD_FLAG'].fillna('N',inplace=True)
ct_df['GRADE'].fillna(5,inplace=True)
ct_df['DUE_DATE'].fillna(200,inplace=True)
ct_df['WIPTURN'].fillna(0,inplace=True)
ct_df['BATCH'].fillna('N',inplace=True)

for i in ['Q_E','Q_P','Q_H','Q_R','Q_W','Q_2','Q_3','Q_4','Q_5','Q_7','Q_8','Q_10']:
    ct_df[i].fillna(0, inplace=True)

print(ct_df.isnull().sum())

DR                 0
PROCESS_ID         0
PART_ID            0
LOTTYPE            0
QTY                0
HOLD_FLAG          0
FLOORID            0
GRADE              0
COMPLETE_RATE      0
LOT_PURPOSE        0
DUE_DATE           0
WIPTURN            0
ROOM               0
PROCESS_GROUP      0
LAYER_TITLE        0
LAYER_GROUP        0
BLOCK_GROUP        0
EQPTYPE            0
MOVE_0             0
EQP_WORKLOAD_0     0
MOVE_1             0
EQP_WORKLOAD_1     0
MOVE_2             0
EQP_WORKLOAD_2     0
BATCH              0
Q_E                0
Q_P                0
L_E                0
L_P                0
Q_H                0
Q_R                0
Q_W                0
L_H                0
L_R                0
L_W                0
Q_2                0
Q_3                0
Q_4                0
Q_5                0
Q_7                0
Q_8                0
Q_10               0
L_2                0
L_3                0
L_4                0
L_5                0
L_7                0
L_8          

In [22]:
# 이상치제거-1 : 타겟기준 상/하한 5% 제거 
print("이상치 제거 전", ct_df.shape)
lower_bound = ct_df['CYCLETIME'].quantile(0.05)
upper_bound = ct_df['CYCLETIME'].quantile(0.95)
ct_df = ct_df[(ct_df['CYCLETIME'] > lower_bound) & (ct_df['CYCLETIME'] < upper_bound)]
print("타겟 데이터 이상치 제거 후", ct_df.shape)

이상치 제거 전 (658097, 57)
타겟 데이터 이상치 제거 후 (587159, 57)


In [23]:
# 이상치제거-② : 숫자형 피쳐에 대해 5시그마 기준 아웃라이어 제거

def filter_outliers(column):
    mean, std = column.mean(), column.std()
    lower_bound, upper_bound = mean - 5 * std, mean + 5 * std
    return column[(column >= lower_bound) & (column <= upper_bound)]

for col in numerical_list:
    ct_df[col] = filter_outliers(ct_df[col])

# 위 코드까지는 outlier를 NULL처리함
ct_df.dropna(inplace=True)

print("피쳐 데이터 이상치 제거 후", ct_df.shape)

피쳐 데이터 이상치 제거 후 (521779, 57)


In [24]:
# 인덱스 초기화
ct_df = ct_df.reset_index(drop=True)

In [25]:
# 모든 숫자형 변수에 대해 z표준화 진행 
scaler = StandardScaler()
ct_df[numerical_list] = scaler.fit_transform(ct_df[numerical_list])
ct_df

,DR,PROCESS_ID,PART_ID,LOTTYPE,QTY,HOLD_FLAG,FLOORID,GRADE,COMPLETE_RATE,LOT_PURPOSE,DUE_DATE,WIPTURN,ROOM,PROCESS_GROUP,LAYER_TITLE,LAYER_GROUP,BLOCK_GROUP,EQPTYPE,MOVE_0,EQP_WORKLOAD_0,MOVE_1,EQP_WORKLOAD_1,MOVE_2,EQP_WORKLOAD_2,BATCH,Q_E,Q_P,L_E,L_P,Q_H,Q_R,Q_W,L_H,L_R,L_W,Q_2,Q_3,Q_4,Q_5,Q_7,Q_8,Q_10,L_2,L_3,L_4,L_5,L_7,L_8,L_10,WIP_WAITTIME,WIP_PRODUCT_NUM,SHIFT_TYPE,PRC_WAIT_MIN,PRC_WAIT_MAX,PRC_WAIT_MEDIAN,PRC_WAIT_STD,CYCLETIME
0,N4,BU,5P9865A02,PT,-1.702672,Y,S3,0.678814,0.338031,J,1.272780,-1.040581,PH,COATER_HSOH,L1,LG61,PU6,PARCCO,0.720655,-0.696233,0.981781,-0.664620,0.707444,-0.684201,N,2.062575,1.735273,3.425849,2.386661,2.447780,0.895343,1.414618,3.472389,0.715582,2.764852,0.919482,1.423341,2.576896,1.137361,0.358854,-0.034845,-0.065732,4.589370,2.841138,3.120002,2.419251,2.512175,2.523029,-0.067493,0.768610,3.398804,WG,-0.365971,4.352058,-0.222606,1.407415,115.0
1,N4,BU,5P9865A02,PT,0.444506,Y,S3,-1.923327,1.156127,1,-1.377403,1.366344,PH,COATER_HSOH,D10,LG63,PU6,PARCCO,0.720655,-0.696233,0.981781,-0.664620,0.707444,-0.684201,N,2.247707,1.729416,3.356999,2.046289,2.937364,0.206645,1.586614,3.424616,0.378586,2.445695,-0.208179,1.838736,2.375083,0.846614,0.436453,0.461473,-0.065732,-0.230689,2.769354,2.646249,2.419251,3.253439,2.893619,-0.067493,0.881273,3.138018,WS,-0.365971,4.352058,-0.222606,1.407415,285.0
2,N5,LJ,FK05M2X01,PP,0.444506,Y,S3,0.678814,0.815253,1,-1.010888,0.244717,PH,COATER_HSOH,D5,LG63,PU6,PARCCO,0.720655,-0.696233,0.981781,-0.664620,0.707444,-0.684201,N,2.247707,1.729416,3.356999,2.046289,2.937364,0.206645,1.586614,3.424616,0.378586,2.445695,-0.208179,1.838736,2.375083,0.846614,0.436453,0.461473,-0.065732,-0.230689,2.769354,2.646249,2.419251,3.253439,2.893619,-0.067493,0.881273,3.138018,WS,-0.148212,0.126315,-0.187582,-0.098518,264.0
3,N5,LJ,FK05M2X01,PP,0.670525,Y,S3,-1.272792,0.235769,1,-0.700760,0.391539,PH,COATER_HSOH,CM,LG51,PU5,PARCCO,0.985945,-0.664618,0.701941,-0.684381,0.790913,-0.684201,N,2.463695,1.550781,3.265198,1.781555,2.633157,1.144039,1.018506,3.376843,0.940246,1.807379,-0.208179,1.772800,1.966763,1.080630,2.376441,0.590516,-0.065732,-0.230689,2.410432,2.527811,2.336403,3.994703,3.140679,-0.067493,0.927776,3.138018,WO,-0.148212,0.126315,-0.187582,-0.098518,379.0
4,N4,BU,5P9865A02,PT,0.557515,Y,S3-D,0.678814,0.781166,1,-1.222339,1.375972,PH,COATER_HSOH,FD,LG63,PU6,PARCCO,0.582931,-0.688329,0.619812,-0.677794,0.615034,-0.681567,N,2.747564,3.442551,3.678301,3.407778,3.541026,2.068680,2.873977,3.854574,1.838902,3.311980,-0.208179,2.906896,3.520256,2.818017,0.436453,0.282799,-0.065732,-0.230689,3.092383,3.830631,3.247732,3.253439,3.017149,-0.067493,0.531556,3.572662,WO,-0.365971,4.352058,-0.222606,1.407415,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521774,N3,C5,FC1978X01,PT,-0.572578,N,S3-E,0.678814,0.201682,U,1.272780,-1.040581,CL,CU_DSPM_BSTRIP_S,X1S,LG61,PU6,WETCCU,-0.906637,-0.804251,-0.927597,-0.796358,-0.922292,-0.796170,N,-0.763778,-0.923747,-0.613374,-0.790147,-0.613306,-1.043215,-0.763997,-0.612217,-0.969399,-0.609100,-0.208179,-0.778915,-0.717824,-0.812768,-0.339542,-0.471606,-0.065732,-0.230689,-0.640405,-0.591059,-0.728976,-0.452879,-0.565222,-0.067493,-0.257346,-1.121492,WD,1.042203,-0.448796,-0.346359,-0.447447,17.0
521775,N5,LJ,FK05M2X01,PP,-1.702672,N,S3,-0.622256,0.815253,1,-0.982695,0.146033,ME,CVD_ENDURA_CO_LINER_EN2VXTAC1,MD,LG63,PU6,MCUMTL,-0.399563,0.502506,-0.644935,0.521025,-0.722530,0.505305,N,-0.769950,-0.786111,-0.636324,-0.638870,-0.608553,-0.749880,-0.769209,-0.588330,-0.632402,-0.654694,-0.208179,-0.785509,-0.600491,-0.656758,-0.339542,-0.471606,-0.065732,-0.230689,-0.676297,-0.512100,-0.563280,-0.452879,-0.565222,-0.067493,-0.563667,-1.034563,HG,-0.402264,-0.446438,-0.413606,-0.516692,45.0
521776,N5,LJ,FK05M2X01,PP,0.783534,N,S3,0.678814,1.360651,1,-1.335113,-

In [26]:
ct_df.to_csv("cycletime_normalized.csv", index=False)